In [1]:
import tensorflow as tf #tensorflow 1.14.0
tf.enable_eager_execution()
print(tf.executing_eagerly())
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
import time
import json
import csv
from glob import glob
from PIL import Image
import pickle
import shutil
from tqdm import tqdm
import chakin
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
import gzip
from gensim import models as gmodel
import subprocess

True


In [2]:
train_annotation_file = '/home/nilay/Documents/coco/captions/annotations/captions_train2014.json'
train_incep_path = '/home/nilay/Documents/coco/train_incep/'

In [3]:
train_incep_paths = glob(train_incep_path+'*/*npy')

print(len(train_incep_paths))

82783


In [4]:
train_incep_paths.sort()
train_incep_paths[-10:]

['/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581204.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581310.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581396.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581507.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581667.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581686.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581697.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581708.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581719.npy',
 '/home/nilay/Documents/coco/train_incep/9/COCO_train2014_000000581882.npy']

In [5]:
train_dict = {}
for f in train_incep_paths:
    train_dict[int(f[56:-4])] = f

In [6]:
train_ids = []
train_paths = []
for key in train_dict:
    train_ids.append(key)
    train_paths.append(train_dict[key])

In [7]:
with open(train_annotation_file, 'r') as f:
    train_annotations = json.load(f)

train_captions = []
train_img_paths = []

for i,annot in tqdm(enumerate(train_annotations['annotations'])):
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    image_path = train_dict[image_id]

    train_img_paths.append(image_path)
    train_captions.append(caption)

train_captions, train_img_paths = shuffle(train_captions,
                                          train_img_paths,
                                          random_state=1)

print(len(train_img_paths))
print(len(list(set(train_img_paths))))

414113it [00:00, 928392.12it/s]


414113
82783


In [8]:
row_word_map = pd.read_csv('/home/nilay/Documents/coco/row_word_map.csv')
embed = np.load('/home/nilay/Documents/coco/embed.npy')

In [9]:
print(row_word_map.shape)
print(embed.shape)

(23899, 3)
(23899, 300)


In [10]:
row_word_map.head()

,Unnamed: 0,row,word
0,0,0,_
1,1,1,a
2,2,2,<start>
3,3,3,<end>
4,4,4,.


In [11]:
embed_dict = {}
for i,row in row_word_map.iterrows():
    row = row.values
    if row[2]!='_':
        embed_dict[row[2]]=row[1]
embed_dict['<pad>'] = 0
embed_dict['<unk>'] = len(embed)

In [12]:
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [13]:
map_df = pd.read_csv('/home/nilay/Documents/coco/corec_map1.csv')
map_dict = {}
for i,row in map_df.iterrows():
    row = row.values
    map_dict[row[0]] = row[1]

In [14]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters="!#$%&()'*+,-./:;=?@[\\]^_`{|}~\t\n"+'"',)
tokenizer.fit_on_texts(train_captions)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
train_seqs = tokenizer.texts_to_sequences(train_captions)

train_text = tokenizer.sequences_to_texts(train_seqs)

train_text_corec = []
for line in train_text:
    line = ' '.join([map_dict[l] if l in map_dict else l for l in line.split()])
    train_text_corec.append(line)

In [15]:
train_embed = []
for line in tqdm(train_text_corec):
    line = [embed_dict[l] if l in embed_dict else embed_dict['<unk>'] for l in line.split()]
    train_embed.append(line)

100%|██████████| 414113/414113 [00:02<00:00, 203457.68it/s]


In [16]:
print(embed.shape)
embed[0] = np.zeros((1,300))
z = np.random.rand(1,300)
embed = np.array(embed.tolist()+z.tolist())
print(embed.shape)

(23899, 300)
(23900, 300)


In [17]:
max_len = calc_max_length(train_seqs)
train_seqs_pad = tf.keras.preprocessing.sequence.pad_sequences(train_embed, padding='post',maxlen=max_len,dtype='int64')
print(max_len)

52


In [18]:
BATCH_SIZE = 128
BUFFER_SIZE = 2000
embedding_dim = 300
units = 600
vocab_size = 23900
num_steps = len(train_img_paths) // BATCH_SIZE
features_shape = 2048
attention_features_shape = 64

In [19]:
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8'))
    return img_tensor, cap

In [20]:
dataset = tf.data.Dataset.from_tensor_slices((train_img_paths, train_seqs_pad))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int64]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [21]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [22]:
class CNN_Encoder(tf.keras.Model):

    def __init__(self, embedding_dim, units):
        super(CNN_Encoder, self).__init__()
        self.units = units

        self.bn = tf.keras.layers.BatchNormalization()
        self.fc = tf.keras.layers.Dense(embedding_dim)
        self.W_h = tf.keras.layers.Dense(self.units)
        self.W_c = tf.keras.layers.Dense(self.units)

    def call(self, x):
        x = self.bn(x)
        x_mean = tf.reduce_mean(x,1)
        h_state = self.W_h(x_mean)
        h_state = tf.nn.tanh(h_state)
        c_state = self.W_c(x_mean)
        c_state = tf.nn.tanh(c_state)

        x = self.fc(x)
        x = tf.nn.relu(x)
        return x ,c_state,h_state

In [23]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,embeddings_initializer=tf.keras.initializers.Constant(embed))
        self.lstm = tf.keras.layers.LSTMCell(self.units)
        self.fc1 = tf.keras.layers.Dense(embedding_dim)
        self.fc2 = tf.keras.layers.Dense(embedding_dim,use_bias=False)
        self.fc3 = tf.keras.layers.Dense(vocab_size)
        self.fc_beta = tf.keras.layers.Dense(1)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, c_state, h_state):
        context_vector, attention_weights = self.attention(features, h_state)

        x = self.embedding(x)

        beta = self.fc_beta(h_state)
        beta = tf.nn.sigmoid(beta)
        context_vector = tf.multiply(beta, context_vector, name='selected_context')

        x_in = tf.concat([x,context_vector], axis=1)

        output,[c_state,h_state] = self.lstm(x,states=[c_state,h_state])

        h_state = tf.nn.dropout(h_state,0.5)
        W_h = self.fc1(h_state)
        W_context = self.fc2(context_vector)
        h_logits = W_h + W_context + x
        h_logits = tf.nn.tanh(h_logits)
        h_logits = tf.nn.dropout(h_logits,0.5)
        out_logits = self.fc3(h_logits)

        return out_logits, c_state, h_state, attention_weights

In [24]:
encoder = CNN_Encoder(embedding_dim, units)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [25]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [26]:
checkpoint_path = '/home/nilay/Documents/coco/checkpoints/train'
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=20)

In [27]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [28]:
loss_plot = []

In [29]:
def train_step(img_tensor, target, epoch):
    loss = 0

    dec_input = tf.convert_to_tensor([tokenizer.word_index['<start>']] * img_tensor.shape[0], 1)
    limit = 0.8 - (epoch/50)

    with tf.GradientTape() as tape:
        features, c_state, h_state = encoder(img_tensor)
        alpha_list = []
        alpha_c = 0.01

        for i in range(1, target.shape[1]):
            predictions, c_state, h_state, attention_weights = decoder(dec_input, features, c_state, h_state)
            alpha_list.append(attention_weights)

            loss += loss_function(target[:, i], predictions)
            predicted_id = tf.argmax(predictions,-1)

            dec_input = tf.cond(tf.random.uniform((1,))<limit,
                  lambda:tf.convert_to_tensor(target[:, i], dtype=tf.int64),
                  lambda:tf.convert_to_tensor(predicted_id, dtype=tf.int64))

        alphas = tf.stack(alpha_list)
        alphas = tf.transpose(alphas, (1, 0, 2,3)) 
        alphas_all = tf.reduce_sum(alphas, 1)   
        alpha_reg = tf.reduce_sum((52./64 - alphas_all) ** 2,1)
        alpha_reg = alpha_c * tf.reduce_mean(alpha_reg)
        loss += alpha_reg

    total_loss = (loss / int(target.shape[1]))

    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss, total_loss

In [ ]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target, epoch)
        total_loss += t_loss

        if batch % 50 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
            epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
          
    loss_plot.append(total_loss / num_steps)

    ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1 Batch 0 Loss 2.2266
Epoch 1 Batch 50 Loss 1.1010
Epoch 1 Batch 100 Loss 1.1060
Epoch 1 Batch 150 Loss 1.0141
Epoch 1 Batch 200 Loss 1.0000
Epoch 1 Batch 250 Loss 0.9029
Epoch 1 Batch 300 Loss 0.9089
Epoch 1 Batch 350 Loss 0.9001
Epoch 1 Batch 400 Loss 0.8689
Epoch 1 Batch 450 Loss 0.7770
Epoch 1 Batch 500 Loss 0.8133
Epoch 1 Batch 550 Loss 0.9422
Epoch 1 Batch 600 Loss 0.8348
Epoch 1 Batch 650 Loss 0.8151
Epoch 1 Batch 700 Loss 0.7677
Epoch 1 Batch 750 Loss 0.8592
Epoch 1 Batch 800 Loss 0.7802
Epoch 1 Batch 850 Loss 0.7044
Epoch 1 Batch 900 Loss 0.8269
Epoch 1 Batch 950 Loss 0.8568
Epoch 1 Batch 1000 Loss 0.8081
Epoch 1 Batch 1050 Loss 0.7512
Epoch 1 Batch 1100 Loss 0.7674
Epoch 1 Batch 1150 Loss 0.8011
Epoch 1 Batch 1200 Loss 0.7436
Epoch 1 Batch 1250 Loss 0.8322
Epoch 1 Batch 1300 Loss 0.7310
Epoch 1 Batch 1350 Loss 0.9121
Epoch 1 Batch 1400 Loss 0.7881
Epoch 1 Batch 1450 Loss 0.7965